In [0]:
# !pip install --upgrade tables
# !pip install xgboost
# !pip install eli5
# !pip install hyperopt

In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

from hyperopt import hp, fmin, tpe, STATUS_OK

import eli5
from eli5 import permutation_importance

In [8]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix_two/dw-predict-car/"

/content/drive/My Drive/Colab Notebooks/dw_matrix_two/dw-predict-car


In [9]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [0]:
SUFFIX_CAT='__cat'

for feat in df.columns.values:
  if isinstance(df[feat][0], list): continue
  
  if SUFFIX_CAT in feat: continue
  df[feat+SUFFIX_CAT] = df[feat].factorize()[0]

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x)=='None' else int(x))
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split(' ')[0]))
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split('cm')[0].replace(' ','')))

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [13]:
feats=['param_napęd__cat','param_rok-produkcji','param_stan__cat','param_skrzynia-biegów__cat','param_faktura-vat__cat','param_moc','param_marka-pojazdu__cat','feature_kamera-cofania__cat','param_typ__cat','seller_name__cat','param_pojemność-skokowa','feature_wspomaganie-kierownicy__cat','param_model-pojazdu__cat','param_wersja__cat','param_kod-silnika__cat','feature_system-start-stop__cat','feature_asystent-pasa-ruchu__cat','feature_czujniki-parkowania-przednie__cat','feature_łopatki-zmiany-biegów__cat','feature_regulowane-zawieszenie__cat']

xgb_params = {
    'max_depth': 5,
    'n_estimators': 50,
    'learning_rate': 0.1,
    'seed': 0
}

model = xgb.XGBRegressor(**xgb_params)
run_model(model, feats)

[21:32:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:33:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:33:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9569.227198767323, 72.83561801421891)

In [16]:
def obj_func(params):
  print("Training with params: ")
  print(params)

  mean_mae, score_std = run_model(xgb.XGBRegressor(**params), feats)

  return {'loss': np.abs(mean_mae), 'status': STATUS_OK}

# space
xgb_reg_params = {
    'learning_rate':      hp.choice('learinig_rate',        np.arange(0.05, 0.61, 0.05)) ,
    'max_depth':          hp.choice('max_depth',            np.arange(5, 20, 1, dtype=int)) ,
    'subsample':          hp.quniform('subsample',          0.5, 1, 0.05 ),
    'colsample_bytree':   hp.quniform('colsample_bytree',   0.5, 1, 0.05 ),
    'objective':          'reg:squarederror',
    'n_estimators':       100,
    'seed':               0
}

# run
best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=50)

Training with params: 
{'colsample_bytree': 0.8500000000000001, 'learning_rate': 0.45, 'max_depth': 19, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 1.0}
Training with params: 
{'colsample_bytree': 0.8500000000000001, 'learning_rate': 0.6000000000000001, 'max_depth': 7, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.7000000000000001}
Training with params: 
{'colsample_bytree': 0.6000000000000001, 'learning_rate': 0.55, 'max_depth': 8, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9}
Training with params: 
{'colsample_bytree': 0.8500000000000001, 'learning_rate': 0.5, 'max_depth': 8, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.6000000000000001}
Training with params: 
{'colsample_bytree': 0.8500000000000001, 'learning_rate': 0.3, 'max_depth': 19, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.6000000000000001}
Training wi